In [55]:
from PyKomoran import *

In [56]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [57]:
dic = pd.read_csv('comment.csv',encoding = 'cp949')
comment_data = dic[['comment']] # 선플 cvs 파일에서 comment만

In [58]:
#한글과 공백을 제외하고 모두 제거
normalized_text = []
for string in comment_data.comment.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 \! \~ \.]+", " ", string) # 문장을 만들 때 문장부호를 사용할 수 있도록 추가해서 학습시킴
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

['정말 노래 대단하네요', '가창력 대단하고.인기도  굉장해요.연예계의 별이 될꺼 같아요', '노래만큼은  정말    넘사벽  맞다.   최고  맞다.  ', '진짜 잘하네용', '노래 정말잘해']


In [59]:
comment_data.comment = normalized_text
for sentence in comment_data['comment']:
    print(sentence)

다니까  내일도 아닌데 다음드라마는 어떻게 쓰려고 저러나 걱정함
작가 진짜대단해유~우
작가 천재 리스펙이당께요 잉~~~  대사 하나하나 버릴께 없다 증말
작가 천재다..... 마지막회까지 완벽하다.... 
작가 필력 최고다!연기자 모두 수고 많으셨어요~
작가. 연출자. 출연배우들 모두모두에게박수를 보냅니다. 보는내내 행복했습니다 ~~~ 
작가가  대라는게 놀랍다
작가가 대사를 모두 명대사로 아주 대단하세요 작가님이 최고네요 누구신지는 모르겠지만
작가가 참 이쁘게 글을 쓰시네요따스해지는 드라마였어요감사합니다
작가님   연기자분들  모두모두 감사했습니다 
작가님  계속 파이팅
작가님  대단하셔오  다음 작품  기대 할게요  그리고  여기 출연하신분들   믿고 보는  배우님들~~
작가님 마지막도 넘최고 였고 흥식이 와~~정말 소름키치게 연기 잘하더라~~ 
작가님  사람들에 대한 따뜻한 시선 참좋았답니다  세상은  착한 사람들이 이끌어가는게 맞는거죠
작가님 감사합니다  두달동안 행복했어요~ 
작가님 감사합니다  해피엔드로 마무리지어주셔서  보는내내 행복했어요
작가님 감사합니다. 대한민국 사람들 가슴깊은곳에 자리잡은 그 깊은 정을 다시 한번 일깨워준 것 같습니다.  극소수인 나쁜 사람들때문에 개개인 스스로가 감정이 매말라가고 냉정해지는 이 시점에서 아 우리는 원래 이렇게 정이 많은 민족이였었어 하고 다시금 알려 주시는 것 같습니다 결국은 쪽수다! 다만 왕따와 조폭등 나쁜일에는 쪽수가 해당되서는 안되겠지요 작가님 대사 하나하나가 정말 주옥같고 되새기게 되네요 간만에 정말 간만에 주말연속극같은 수목 드라마를 만나게 된 것 같아요  년에 한번 나올까 말까네요 매번 바람 불륜 살인 친자확인
작가님 감사합니다. 이런드라마 보게해주셔서 
작가님 감사합니다.기적같은 해피엔딩이라 눈물머금고 미소지으며 그리고 박수치며 봤어요.정말 좋은 드라마 였어요.
작가님 감사합니다.따뜻해요
작가님 고맙습니다~ 늘 건강하세요~
작가님 대단하시고 공효진언니가 까불이직접 잡는거 대박~~좋은배우들 다 좋

In [60]:
stopwords=['의','가','이','은','들','는','가수','연기','걍','과','도','를','으로','자','에','와','한','하다','배우','가수','와','배그','박지훈','연우','드라마','작가','노래']#불용어 제거하기

# 토큰화

In [61]:
kmr = Komoran("EXP")
X_token=[]
for sentence in comment_data['comment']:
    temp_X = []
    temp_X = kmr.morphes(sentence)
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_token.append(temp_X)

In [62]:
max_words = 20000
t = Tokenizer(num_words = max_words) # 상위 20,000개의 단어만 보존
t.fit_on_texts(X_token)

# 정수 인코딩 수행

In [63]:
sequences = list()

for line in comment_data['comment']: # 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        for j in range(1, i):
            sequence = encoded[j:i+1]
            sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[227, 53],
 [227, 53, 48],
 [53, 48],
 [227, 53, 48, 3],
 [53, 48, 3],
 [48, 3],
 [1566, 480],
 [1071, 607],
 [1071, 607, 63],
 [607, 63],
 [63, 63]]

In [64]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

In [65]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 19


In [66]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 227
   53]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 227  53
   48]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  53
   48]]


In [67]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [68]:
vocab_size = len(t.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)

# 문장 생성

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [70]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=60)

Train on 11025 samples
Epoch 1/60
11025/11025 [==============================] - 28s 3ms/sample - loss: 6.1435 - accuracy: 0.0445
Epoch 2/60
11025/11025 [==============================] - 19s 2ms/sample - loss: 5.6938 - accuracy: 0.0500
Epoch 3/60
11025/11025 [==============================] - 18s 2ms/sample - loss: 5.6183 - accuracy: 0.0493
Epoch 4/60
11025/11025 [==============================] - 19s 2ms/sample - loss: 5.5467 - accuracy: 0.0493
Epoch 5/60
11025/11025 [==============================] - 18s 2ms/sample - loss: 5.4578 - accuracy: 0.0521
Epoch 6/60
11025/11025 [==============================] - 19s 2ms/sample - loss: 5.3577 - accuracy: 0.0580
Epoch 7/60
11025/11025 [==============================] - 17s 2ms/sample - loss: 5.2642 - accuracy: 0.0655
Epoch 8/60
11025/11025 [==============================] - 19s 2ms/sample - loss: 5.1751 - accuracy: 0.0703
Epoch 9/60
11025/11025 [==============================] - 19s 2ms/sample - loss: 5.0949 - accuracy: 0.0772
Epoch 10/60
11

In [71]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=18, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [72]:
print(sentence_generation(model, t, '너무', 20))
# 임의의 단어에 대해서 10개의 단어를 추가 생성

너무 정말 정말 정말 콘서트 정말 정말 정말 콘서트 정말 정말 정말 콘서트 정말 정말 정말 콘서트 정말 정말 정말 콘서트
